In [ ]:
!pip install -r requirements.txt

In [ ]:
import math
from os import path, makedirs

import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib import cm, colors

INPUT_SHP = path.join("data", "kuusiku_data.shp")
OUTPUT_CSV = path.join("model_simulation_results.csv")
OUTPUT_IMAGE = path.join("output_plot.png")

# Constants for the model
V_MAX_0 = V_MAX_UPTAKE_0 = 100000000
EA = EA_UPTAKE = 47
GAS_CONST = 0.008314
KM_UPTAKE_SLOPE = 0.01
KM_UPTAKE_0 = 0.1
INPUT_SOC = 0.0005
MIC_TO_SOC = 0.5
KM_SLOPE = 5
KM_0 = 500
CUE_SLOPE = -0.016
CUE_0 = 0.63
R_ENZ_LOSS = 0.001
R_DEATH = 0.0002
R_ENZ_PROD = 0.000005


def calculate_microbial_biomass_changes(assim, cue, death, eprod):
    return (assim * cue) - death - eprod


def calculate_assimilation(v_max_uptake, mic, doc, km_uptake):
    return v_max_uptake * mic * (doc / (km_uptake + doc))


def calculate_death(mic):
    return R_DEATH * mic


def calculate_eprod(mic):
    return R_ENZ_PROD * mic


def calculate_v_max_uptake(temperature):
    return V_MAX_UPTAKE_0 * math.exp(-1 * (EA_UPTAKE / (GAS_CONST * (temperature + 273))))


def calculate_km_uptake(temperature):
    return (KM_UPTAKE_SLOPE * temperature) + KM_UPTAKE_0


def calculate_carbon_use_efficiency(temperature, acclimation=False):
    if acclimation:
        cue_slope = CUE_SLOPE / 2  # Reduced sensitivity under warming
    else:
        cue_slope = CUE_SLOPE
    return (cue_slope * temperature) + CUE_0


def enzyme_pool_increase(eprod, eloss):
    return eprod - eloss


def calculate_enzyme_loss(enz):
    return R_ENZ_LOSS * enz


def calculate_soil_organic_carbon(death, decomp):
    return INPUT_SOC + (death * MIC_TO_SOC) - decomp


def calculate_v_max(temperature, acclimation=False):
    if acclimation:
        v_max_0 = V_MAX_0 / 2  # Reduced sensitivity under warming
    else:
        v_max_0 = V_MAX_0
    return v_max_0 * math.exp(-1 * (EA / (GAS_CONST * (temperature + 273))))


def calculate_km(temperature):
    return (KM_SLOPE * temperature) + KM_0


def calculate_doc(death, decomp, eloss, assim):
    return INPUT_SOC + (death * (1 - MIC_TO_SOC)) + decomp + eloss - assim


def simulate_model(mic, enz, soc, doc):
    # Create an empty DataFrame to store data
    data = pd.DataFrame(columns=["Time", "SOC", "DOC", "Microbial Biomass", "Extracellular Enzymes"])

    # Simulate warming from 20deg to 25deg (SIMULATION PURPOSES ONLY)
    # Todo: Replace these hardcoded values with parameters to be inserted into the model.
    initial_temperature = 20
    final_temperature = 25

    # Simulate constant CUE (no acclimation)
    cue_constant = calculate_carbon_use_efficiency(initial_temperature, acclimation=False)

    # Simulate changes over time (SIMULATION PURPOSES ONLY)
    # Todo: Replace this hardcoded value with a parameter to be inserted into the model.
    time_steps = 70

    temperature_change = (final_temperature - initial_temperature) / time_steps

    for time_step in range(time_steps):
        # Calculate parameters based on the current temperature in calculation
        temperature = initial_temperature + temperature_change
        v_max_uptake = calculate_v_max_uptake(temperature)
        km_uptake = calculate_km_uptake(temperature)

        # Calculate assimilation, death, enzyme production, enzyme loss
        assimilation = calculate_assimilation(v_max_uptake, mic, doc, km_uptake)
        death = calculate_death(mic)
        eprod = calculate_eprod(mic)
        eloss = calculate_enzyme_loss(enz)

        # Calculate changes in pool sizes
        mic_change = calculate_microbial_biomass_changes(assimilation, cue_constant, death, eprod)
        enz_change = enzyme_pool_increase(eprod, eloss)
        soc_change = calculate_soil_organic_carbon(death, 0)  # No decomposition assumed
        doc_change = calculate_doc(death, 0, eloss, assimilation)

        # Update pool sizes for the next time step
        mic += mic_change
        enz += enz_change
        soc += soc_change
        doc += doc_change

        # Append the data to the DataFrame
        data = data._append(
            {
                "Time": time_step + 1,
                "SOC": soc,
                "DOC": doc,
                "Microbial Biomass": mic,
                "Extracellular Enzymes": enz,
            },
            ignore_index=True)

    # Todo: The DataFrame could be saved to a file here for deeper analysis at this point.
    # For now, just return the last updated values
    return soc, enz, mic, doc


def read_shp_data():
    print("Reading in shapefile data: %s" % INPUT_SHP)
    data = gpd.read_file(INPUT_SHP)
    return data


def calculate_predicted_values(data):
    print("Processing values for %s plots" % len(data))
    predicted_soc_values = pd.Series()
    for index, row in data.iterrows():
        soc = row['soc']
        mic = row['mic']
        enz = row['enz']
        doc = row['doc']
        predicted_soc, predicted_enz, predicted_mic, predicted_doc = simulate_model(mic, enz, soc, doc)
        predicted_soc_values[index] = predicted_soc
        # Todo: handle other three variables
    # Add the predicted values as a new column to the GeoDataFrame
    data['predicted_soc'] = predicted_soc_values


def write_results_to_csv(data):
    print("Writing results to csv file: %s" % OUTPUT_CSV)
    csv_data = data[['id', 'soc', 'predicted_soc']]
    csv_data.to_csv(OUTPUT_CSV, index=False)


def plot_results(data):
    print("Generating predicition image for SOC: %s" % OUTPUT_IMAGE)
    v_min = min(data['soc'].min(), data['predicted_soc'].min())
    v_max = max(data['soc'].max(), data['predicted_soc'].max())

    # Create two subplots
    fig, ax = plt.subplots(1, 2, figsize=(12, 6))

    # Plot the 'soc' values on the left axis
    data.plot(column='soc', cmap='OrRd', linewidth=0.8, ax=ax[0], edgecolor='0.8', legend=False, vmin=v_min, vmax=v_max)
    ax[0].set_title('SOC')
    ax[0].axis('off')

    # Plot the 'predicted_soc' values on the right axis
    data.plot(column='predicted_soc', cmap='OrRd', linewidth=0.8, ax=ax[1], edgecolor='0.8', legend=False, vmin=v_min,
              vmax=v_max)
    ax[1].set_title('Predicted SOC')
    ax[1].axis('off')

    # Create a color bar for both plots
    normalized_colors = colors.Normalize(vmin=v_min, vmax=v_max)
    n_cmap = cm.ScalarMappable(norm=normalized_colors, cmap='OrRd')
    n_cmap.set_array([])

    # Add space for color bar to the right
    fig.subplots_adjust(right=0.85)
    cbar_ax = fig.add_axes([0.88, 0.15, 0.04, 0.7])
    colorbar = fig.colorbar(n_cmap, cax=cbar_ax)
    colorbar.set_label('Soil Organic Carbon (mg g^-1)')

    # Save the plot
    fig.suptitle('SOC Prediction Plot for 70 Time Units (Kuusiku, Estonia)')
    plt.show()


def main():
    print("==================================================")
    print("WELCOME TO MODEL SIMULATOR (AUTHOR: DR. KAI-YUN LI")
    print("==================================================")
    data = read_shp_data()
    calculate_predicted_values(data)
    write_results_to_csv(data)
    plot_results(data)
    print("Done")


if __name__ == '__main__':
    main()
